# <center>The Camera LookAt() Function</center>

I am trying to follow the tutorial on the LearnOpenGL website, which has been pretty good so far, but early on I decided (foolishly maybe?) to not use the GLM libraries, and instead go with the Eigen linear algebra libraries.  Eigen seemed a bit more general purpose, very fast, supported SIMD, and it seemed generally to be a good library to know how to use.  But of course that meant that the canned function for creating the view matrix, glm::lookAt(), was not available, and I would need to create one.

This was just fine with me, as the discussion of the algorithms was right there in the tutorial.  But somehow it was not working as I expected.  Examining the matrix side-by-side with the GLM version was not producing consistent results, and of course the view did not produce anything I was expecting when used with the shader.

Like most people, I did a web search on something like 'Eigen lookat function', and found a couple examples that seemed to work like the GLM version.  That's just fine, but I really wanted to know what was going on with the tutorial.  So I decided to play around with the functions, and the easiest way I know to play around with something like this is in numpy.

Here we go...

In [1]:
import sys, os
import numpy as np

In [4]:
def normalized(a, axis=-1, order=2):
    '''
        Annoying: numpy.linalg.norm() does not do a vector normalization,
                  but a Frobenius norm, which is essentially the
                  Euclidian length.  This is the best solution I have
                  found for normalizing a vector.
    '''
    len_a = np.atleast_1d(np.linalg.norm(a, order, axis))
    len_a[len_a==0] = 1

    return a / np.expand_dims(len_a, axis)

def lookat_view_v1(position, target, up):
    '''
        OK, this is basically how our camera class
        is currently working.
    '''
    camera_direction = normalized(position - target)
    camera_right = normalized(np.cross(up, camera_direction))
    camera_up = np.cross(camera_direction, camera_right)

    view = np.identity(4)
    view[:3, 0] = camera_right
    view[:3, 1] = camera_up
    view[:3, 2] = camera_direction

    view[3, 0] = -camera_right.dot(position)
    view[3, 1] = -camera_up.dot(position)
    view[3, 2] = -camera_direction.dot(position)

    return view


def lookat_view_v2(position, target, up):
    '''
        Here is basically the same thing, but
        with a slightly different strategy.
    '''
    camera_direction = normalized(position - target)
    camera_right = normalized(np.cross(up, camera_direction))
    camera_up = np.cross(camera_direction, camera_right)

    R = np.identity(3)
    R[:, 0] = camera_right
    R[:, 1] = camera_up
    R[:, 2] = camera_direction

    view = np.identity(4)
    view[:3, :3] = R
    view[3, :3] = -R.T.dot(position)
    
    return view


position = np.array([3.0, 0.0, 3.0])
target = np.array([0.0, 0.0, 0.0])
up = np.array([0.0, 1.0, 0.0])
print "position: ", position
print "target: ", target
print "up: ", up

print
print lookat_view_v1(position, target, up).T
print
print lookat_view_v2(position, target, up).T


position:  [ 3.  0.  3.]
target:  [ 0.  0.  0.]
up:  [ 0.  1.  0.]

[[ 0.70710678  0.         -0.70710678 -0.        ]
 [-0.          1.          0.         -0.        ]
 [ 0.70710678  0.          0.70710678 -4.24264069]
 [ 0.          0.          0.          1.        ]]

[[ 0.70710678  0.         -0.70710678 -0.        ]
 [-0.          1.          0.         -0.        ]
 [ 0.70710678  0.          0.70710678 -4.24264069]
 [ 0.          0.          0.          1.        ]]


This seems to be giving a view matrix that is consistent with my C++ Camera.  And these are more or less the same results I get when using the glm::lookAt() function.

But we got these two methods from some stackoverflow code samples.  I would really like to know if the tutorial
on the LearnOpenGL website is going to come up with the same thing.

The LearnOpenGL tutorial shows a formula representing a dot product of an affine angle transform and an affine position transform, so this is precisely what we will try to do.

$$
LookAt = 
\left[
\begin{matrix}
    \color{red}{R_x} & \color{red}{R_y} & \color{red}{R_z} & 0 \\
    \color{green}{U_x} & \color{green}{U_y} & \color{green}{U_z} & 0 \\
    \color{blue}{D_x} & \color{blue}{D_y} & \color{blue}{D_z} & 0 \\
    0 & 0 & 0 & 1 \\
\end{matrix}
\right]
\cdot
\left[
\begin{matrix}
    1 & 0 & 0 & \color{purple}{-P_x} \\
    0 & 1 & 0 & \color{purple}{-P_y} \\
    0 & 0 & 1 & \color{purple}{-P_z} \\
    0 & 0 & 0 & 1
\end{matrix}
\right]
$$

In [5]:
def lookat_view_v3(position, target, up):
    '''
        Here is basically the same thing, but
        going as closely as possible to the tutorial
        on learnopengl.com.
    '''
    camera_direction = normalized(position - target)
    camera_right = normalized(np.cross(up, camera_direction))
    camera_up = np.cross(camera_direction, camera_right)

    view = np.identity(4)
    view[:3, 0] = camera_right
    view[:3, 1] = camera_up
    view[:3, 2] = camera_direction

    position_mx = np.identity(4)
    position_mx[3, :3] = -position

    return view.dot(position_mx)


print lookat_view_v3(position, target, up).T


[[ 0.70710678  0.         -0.70710678 -3.        ]
 [ 0.          1.          0.          0.        ]
 [ 0.70710678  0.          0.70710678 -3.        ]
 [ 0.          0.          0.          1.        ]]


Well, it doesn't appear to be the same.  The angles are correct, which is expected since we don't really change those formulas.  But the positional coefficients are quite different.  The only thing the tutorial states about the view coefficients is that <i>"$\color{red}{R}$ is the right vector, $\color{green}{U}$ is the up vector, $\color{blue}{D}$ is the direction vector and $\color{purple}{P}$ is the camera's position vector"</i>

Some online discussions have given me what I think is some more insight into this.  Basically, the vector $\color{purple}{P}$ is the camera position, but it has been translated by our rotational matrix.  This was not stated, or obviously stated, on the LearnOpenGL site.

So we define the rotated position vector $\color{purple}{P'}$ as:

$$
\color{purple}{P'} = 
\left[
\begin{matrix}
    \color{red}{R_x} & \color{red}{R_y} & \color{red}{R_z} \\
    \color{green}{U_x} & \color{green}{U_y} & \color{green}{U_z} \\
    \color{blue}{D_x} & \color{blue}{D_y} & \color{blue}{D_z}
\end{matrix}
\right]
\cdot
\left[
\begin{matrix}
    \color{purple}{P_x} \\
    \color{purple}{P_y} \\
    \color{purple}{P_z}
\end{matrix}
\right]
$$

And then our LookAt transform becomes:

$$
\begin{align}
    LookAt &= 
    \left[
    \begin{matrix}
        \color{red}{R_x} & \color{red}{R_y} & \color{red}{R_z} & 0 \\
        \color{green}{U_x} & \color{green}{U_y} & \color{green}{U_z} & 0 \\
        \color{blue}{D_x} & \color{blue}{D_y} & \color{blue}{D_z} & 0 \\
        0 & 0 & 0 & 1 \\
    \end{matrix}
    \right]
    \cdot
    \left[
    \begin{matrix}
        1 & 0 & 0 & \color{purple}{-P'_x} \\
        0 & 1 & 0 & \color{purple}{-P'_y} \\
        0 & 0 & 1 & \color{purple}{-P'_z} \\
        0 & 0 & 0 & 1
    \end{matrix}
    \right] \\
    &=
    \left[
    \begin{matrix}
        \color{red}{R_x} & \color{red}{R_y} & \color{red}{R_z} & \color{purple}{-P'_x} \\
        \color{green}{U_x} & \color{green}{U_y} & \color{green}{U_z} & \color{purple}{-P'_y} \\
        \color{blue}{D_x} & \color{blue}{D_y} & \color{blue}{D_z} & \color{purple}{-P'_z} \\
        0 & 0 & 0 & 1 \\
    \end{matrix}
    \right]    
\end{align}
$$

This can of course be expressed in various ways, but I believe the description on the LearnOpenGL site is either incorrect, or maybe it is simply vague in describing its treatment of $\color{purple}{P}$.

Of course the tutorial doesn't actually program the function, but uses the one supplied by GLM.  So I'm sure most people who went through it didn't need to deeply consider the veracity of the formulas given.